In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import re
import requests
import scipy as sp


In [2]:
# GPCRdb data
GPCRdb_fil = pd.read_csv("gpcr_filtered_mutants_0217.csv")
GPCRdb_fil


,Unnamed: 0,reference,protein,mutation_pos,mutation_from,mutation_to,ligand_name,ligand_idtype,ligand_id,ligand_class,...,exp_mu_effect_value,exp_fold_change,exp_mu_effect_qual,exp_mu_effect_ligand_prop,exp_mu_ligand_ref,opt_receptor_expression,opt_basal_activity,opt_gain_of_activity,opt_ligand_emax,opt_agonist
0,22,15626750,hrh1_human,37,I,V,Cetirizine,ChEMBL Compound ID,CHEMBL1000,Binding - unknown pharmacological activity,...,7.700,-1.585,NaN,NaN,[3H]-mepyramine (radioligand),94.2,0.0,NaN,0.0,NaN
1,23,15626750,hrh1_human,38,C,S,Cetirizine,ChEMBL Compound ID,CHEMBL1000,Binding - unknown pharmacological activity,...,7.900,1.000,NaN,NaN,[3H]-mepyramine (radioligand),141.3,0.0,NaN,0.0,NaN
2,24,15626750,hrh1_human,84,N,S,Cetirizine,ChEMBL Compound ID,CHEMBL1000,Binding - unknown pharmacological activity,...,7.500,-2.513,NaN,NaN,[3H]-mepyramine (radioligand),53.7,0.0,NaN,0.0,NaN
3,26,16408006,hrh1_human,111,S,A,Cetirizine,ChEMBL Compound ID,CHEMBL1000,Binding - unknown pharmacological activity,...,7.300,1.000,NaN,NaN,[3H]-mepyramine (radioligand),58.3,0.0,NaN,0.0,NaN
4,27,16408006,hrh1_human,111,S,C,Cetirizine,ChEMBL Compound ID,CHEMBL1000,Binding - unknown pharmacological activity,...,7.500,1.585,NaN,NaN,[3H]-mepyramine (radioligand),55.0,0.0,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11974,455,10.1124/mol.63.5.1021,aa3r_human,274,N,A,VUF-5455,PubChem CID,10339071,Allosteric inverse agonist,...,0.083,-3.322,NaN,NaN,I-AB-MECA,0.0,0.0,=,0.0,NaN
11975,456,10.1124/mol.63.5.1021,aa3r_human,274,N,A,I-AB-MECA,PubChem CID,44208896,Full agonist,...,0.085,-1.328,NaN,NaN,NaN,0.0,0.0,=,0.0,NaN
11976,457,10.1006/bbrc.2001.5027,aa3r_human,282,Y,F,Cl-IB-MECA,PubChem CID,393593,Full agonist,...,27.600,-29.412,NaN,NaN,NaN,0.0,0.0,=,0.0,NaN
11977,458,10.1006/bbrc.2001.5027,aa3r_human,282,Y,F,I-AB-MECA,PubChem CID,44208896,Full agonist,...,9.300,-7.752,NaN,NaN,NaN,0.0,0.0,=,0.0,NaN


In [3]:
#Dynamut2 prediction
dynamut_pred = pd.read_csv("dynamut2_out.csv")
# rename the column EntryName for merging
dynamut_pred = dynamut_pred.rename(columns={'prediction':'Dynamut2'})
dynamut_pred


,Unnamed: 0,mutation_from,mutation_pos,mutation_to,mutation,chain,Dynamut2,Uniprot,protein,gene
0,0,L,340,A,L340A,A,-1.20,P07550,adrb2_human,ADRB2
1,1,S,204,C,S204C,A,-0.21,P07550,adrb2_human,ADRB2
2,2,Y,326,F,Y326F,A,-0.14,P07550,adrb2_human,ADRB2
3,3,S,203,C,S203C,A,-0.46,P07550,adrb2_human,ADRB2
4,4,L,272,C,L272C,A,-1.29,P07550,adrb2_human,ADRB2
...,...,...,...,...,...,...,...,...,...,...
2480,2480,W,196,A,W196A,A,-2.14,Q9Y5N1,hrh3_human,HRH3
2481,2481,T,204,A,T204A,A,-0.57,Q9Y5N1,hrh3_human,HRH3
2482,2482,A,122,V,A122V,A,0.93,Q9Y5N1,hrh3_human,HRH3
2483,2483,L,199,A,L199A,A,-2.03,Q9Y5N1,hrh3_human,HRH3


In [4]:
# merge two dataframes
gpcr_dyna  = pd.merge(GPCRdb_fil,dynamut_pred,on=['mutation_from','mutation_pos','mutation_to','protein'])
gpcr_dyna

,Unnamed: 0_x,reference,protein,mutation_pos,mutation_from,mutation_to,ligand_name,ligand_idtype,ligand_id,ligand_class,...,opt_basal_activity,opt_gain_of_activity,opt_ligand_emax,opt_agonist,Unnamed: 0_y,mutation,chain,Dynamut2,Uniprot,gene
0,22,15626750,hrh1_human,37,I,V,Cetirizine,ChEMBL Compound ID,CHEMBL1000,Binding - unknown pharmacological activity,...,0.0,NaN,0.0,NaN,1579,I37V,A,-1.12,P35367,HRH1
1,78,15626750,hrh1_human,37,I,V,HP-HA,ChEMBL Compound ID,CHEMBL264491,Binding - unknown pharmacological activity,...,0.0,NaN,0.0,NaN,1579,I37V,A,-1.12,P35367,HRH1
2,106,15626750,hrh1_human,37,I,V,[3H]-mepyramine,ChEMBL Compound ID,CHEMBL511,Binding - unknown pharmacological activity,...,0.0,NaN,0.0,NaN,1579,I37V,A,-1.12,P35367,HRH1
3,164,15626750,hrh1_human,37,I,V,Histamine,ChEMBL Compound ID,CHEMBL90,Binding - unknown pharmacological activity,...,0.0,NaN,0.0,NaN,1579,I37V,A,-1.12,P35367,HRH1
4,210,15626750,hrh1_human,37,I,V,HP-HP,SMILES,C1(CCNCCC2=CNC(CCC(C3=CC=CC=C3)C4=CC=CC=C4)=N2...,Binding - unknown pharmacological activity,...,0.0,NaN,0.0,NaN,1579,I37V,A,-1.12,P35367,HRH1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9704,437,10.1021/jm050968b,aa3r_human,272,H,D,CID75022483,PubChem CID,75022483,Full agonist,...,0.0,=,0.0,NaN,226,H272D,A,-1.80,P0DMS8,ADORA3
9705,439,10.1021/jm050968b,aa3r_human,272,H,D,CHEMBL377071,ChEMBL Compound ID,377071,Full agonist,...,0.0,=,0.0,NaN,226,H272D,A,-1.80,P0DMS8,ADORA3
9706,457,10.1006/bbrc.2001.5027,aa3r_human,282,Y,F,Cl-IB-MECA,PubChem CID,393593,Full agonist,...,0.0,=,0.0,NaN,213,Y282F,A,-0.43,P0DMS8,ADORA3
9707,458,10.1006/bbrc.2001.5027,aa3r_human,282,Y,F,I-AB-MECA,PubChem CID,44208896,Full agonist,...,0.0,=,0.0,NaN,213,Y282F,A,-0.43,P0DMS8,ADORA3


In [6]:
gpcr_dyna['abs_fc'] = abs(gpcr_dyna['exp_fold_change'])
gpcr_dyna

,Unnamed: 0_x,reference,protein,mutation_pos,mutation_from,mutation_to,ligand_name,ligand_idtype,ligand_id,ligand_class,...,opt_gain_of_activity,opt_ligand_emax,opt_agonist,Unnamed: 0_y,mutation,chain,Dynamut2,Uniprot,gene,abs_fc
0,0,8903934,hrh1_human,194,T,A,(S)-cetirizine,ChEMBL Compound ID,CHEMBL1334217,Binding - unknown pharmacological activity,...,NaN,0.0,NaN,1951,T194A,A,-0.19,P35367,HRH1,10.000
1,8,8198587,hrh1_human,194,T,A,2-methylhistamine,ChEMBL Compound ID,CHEMBL12620,Binding - unknown pharmacological activity,...,NaN,0.0,NaN,1951,T194A,A,-0.19,P35367,HRH1,1.681
2,12,11809864,hrh1_human,194,T,A,(S)-cetirizine,ChEMBL Compound ID,CHEMBL1334217,Binding - unknown pharmacological activity,...,NaN,0.0,NaN,1951,T194A,A,-0.19,P35367,HRH1,12.658
3,19,8093027,hrh1_human,194,T,A,[3H]-mepyramine,ChEMBL Compound ID,CHEMBL511,Binding - unknown pharmacological activity,...,NaN,0.0,NaN,1951,T194A,A,-0.19,P35367,HRH1,3.155
4,20,8198587,hrh1_human,194,T,A,Histamine,ChEMBL Compound ID,CHEMBL90,Binding - unknown pharmacological activity,...,NaN,0.0,NaN,1951,T194A,A,-0.19,P35367,HRH1,2.295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11709,438,10.1006/bbrc.2001.5027,aa3r_human,107,D,R,I-AB-MECA,PubChem CID,44208896,Full agonist,...,=,0.0,NaN,265,D107R,A,-0.96,P0DMS8,ADORA3,1.395
11710,233,10.1021/jm050968b,aa3r_human,94,T,E,CID85116573,PubChem CID,85116573,Full agonist,...,=,0.0,NaN,268,T94E,A,-0.70,P0DMS8,ADORA3,1.053
11711,271,10.1021/jm050968b,aa3r_human,94,T,E,CHEMBL377071,ChEMBL Compound ID,377071,Full agonist,...,=,0.0,NaN,268,T94E,A,-0.70,P0DMS8,ADORA3,1.130
11712,279,10.1021/jm050968b,aa3r_human,94,T,E,CID75022483,PubChem CID,75022483,Full agonist,...,=,0.0,NaN,268,T94E,A,-0.70,P0DMS8,ADORA3,1.518


In [5]:
gpcr_dyna.to_csv('GPCR_Dynamut2_df.csv',index=False)

In [7]:
gpcr_dyna = pd.read_csv("GPCR_Dynamut2_df.csv")
gpcr_dyna

,Unnamed: 0,Unnamed: 0_x,reference,protein,mutation_pos,mutation_from,mutation_to,ligand_name,ligand_idtype,ligand_id,...,opt_gain_of_activity,opt_ligand_emax,opt_agonist,Unnamed: 0_y,mutation,chain,Dynamut2,Uniprot,gene,abs_fc
0,0,0,8903934,hrh1_human,194,T,A,(S)-cetirizine,ChEMBL Compound ID,CHEMBL1334217,...,NaN,0.0,NaN,1951,T194A,A,-0.19,P35367,HRH1,10.000
1,1,8,8198587,hrh1_human,194,T,A,2-methylhistamine,ChEMBL Compound ID,CHEMBL12620,...,NaN,0.0,NaN,1951,T194A,A,-0.19,P35367,HRH1,1.681
2,2,12,11809864,hrh1_human,194,T,A,(S)-cetirizine,ChEMBL Compound ID,CHEMBL1334217,...,NaN,0.0,NaN,1951,T194A,A,-0.19,P35367,HRH1,12.658
3,3,19,8093027,hrh1_human,194,T,A,[3H]-mepyramine,ChEMBL Compound ID,CHEMBL511,...,NaN,0.0,NaN,1951,T194A,A,-0.19,P35367,HRH1,3.155
4,4,20,8198587,hrh1_human,194,T,A,Histamine,ChEMBL Compound ID,CHEMBL90,...,NaN,0.0,NaN,1951,T194A,A,-0.19,P35367,HRH1,2.295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11709,11709,438,10.1006/bbrc.2001.5027,aa3r_human,107,D,R,I-AB-MECA,PubChem CID,44208896,...,=,0.0,NaN,265,D107R,A,-0.96,P0DMS8,ADORA3,1.395
11710,11710,233,10.1021/jm050968b,aa3r_human,94,T,E,CID85116573,PubChem CID,85116573,...,=,0.0,NaN,268,T94E,A,-0.70,P0DMS8,ADORA3,1.053
11711,11711,271,10.1021/jm050968b,aa3r_human,94,T,E,CHEMBL377071,ChEMBL Compound ID,377071,...,=,0.0,NaN,268,T94E,A,-0.70,P0DMS8,ADORA3,1.130
11712,11712,279,10.1021/jm050968b,aa3r_human,94,T,E,CID75022483,PubChem CID,75022483,...,=,0.0,NaN,268,T94E,A,-0.70,P0DMS8,ADORA3,1.518


In [8]:
value = 'exp_fold_change'
value1 = 'abs_fc'
vep = 'Dynamut2'

In [9]:
p_df1 = gpcr_dyna.groupby(by=["protein","mutation_pos","mutation_from"]).max()[[vep,value]]
p_df2 = gpcr_dyna.groupby(by=["protein","mutation_pos","mutation_from"]).min()[[vep,value]]
p_df3 = gpcr_dyna.groupby(by=["protein","mutation_pos","mutation_from"]).mean()[[vep,value1]]
#dataframe only include positive exp_fc
p_df_pos = gpcr_dyna[gpcr_dyna[value]>0]
#dataframe only include negative exp_fc
p_df_neg = gpcr_dyna[gpcr_dyna[value]<0]

# average of positive exp_fc
p_df4 = p_df_pos.groupby(by=["protein","mutation_pos","mutation_from"]).mean()[[vep,value]] 
# average of negative exp_fc
p_df5 = p_df_neg.groupby(by=["protein","mutation_pos","mutation_from"]).mean()[[vep,value]]

/var/folders/d_/0pdh465d0yj2_tbpjd8dr11c0000gn/T/ipykernel_37267/4066727645.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  p_df1 = gpcr_dyna.groupby(by=["protein","mutation_pos","mutation_from"]).max()[[vep,value]]
/var/folders/d_/0pdh465d0yj2_tbpjd8dr11c0000gn/T/ipykernel_37267/4066727645.py:2: FutureWarning: Dropping invalid columns in DataFrameGroupBy.min is deprecated. In a future version, a TypeError will be raised. Before calling .min, select only columns which should be valid for the function.
  p_df2 = gpcr_dyna.groupby(by=["protein","mutation_pos","mutation_from"]).min()[[vep,value]]


In [10]:
print("max(fc) with ddg:")
print(sp.stats.spearmanr(p_df1[value], p_df1[vep]))
print("\n")
print("min(fc) with ddg:")
print(sp.stats.spearmanr(p_df2[value], p_df2[vep]))
print("\n")
print("mean(abs(fc)) with ddg:")
print(sp.stats.spearmanr(p_df3[value1], p_df3[vep]))
print("\n")

print("mean(posfc) with ddg:")
print(sp.stats.spearmanr(p_df4[value], p_df4[vep]))
print("\n")
print("mean(negfc) with ddg:")
print(sp.stats.spearmanr(p_df5[value], p_df5[vep]))


max(fc) with ddg:
SpearmanrResult(correlation=-0.06353845575817256, pvalue=0.002725619176495101)


min(fc) with ddg:
SpearmanrResult(correlation=0.05898168182401226, pvalue=0.00540637660681801)


mean(abs(fc)) with ddg:
SpearmanrResult(correlation=-0.1539347492688852, pvalue=2.936554370559587e-13)


mean(posfc) with ddg:
SpearmanrResult(correlation=-0.15778657019639156, pvalue=4.015956396659575e-12)


mean(negfc) with ddg:
SpearmanrResult(correlation=0.12184034104495581, pvalue=2.3523888580566584e-05)


## try another option: classify into three groups---accuracy, sensitivity, specificity

In [8]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report, roc_curve, auc, confusion_matrix

In [25]:
# ddg into 3 classifications: (neg infinity,-1), [-1,1],(1,pos infinity)
gpcr_dyna['Dynamut2_group'] = gpcr_dyna['Dynamut2'].apply(lambda x: 1 if x < -1 else (1 if x > 1 else 0))
gpcr_dyna['foldchange_group'] = gpcr_dyna['exp_fold_change'].apply(lambda x: 1 if x < -5 else (1 if x > 5 else 0))


In [26]:
true_labels = gpcr_dyna['foldchange_group']
predicted_labels = gpcr_dyna['Dynamut2_group']



In [27]:
# Calculate confusion matrix
tn, fp, fn, tp = confusion_matrix(true_labels, predicted_labels).ravel()

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)

# Calculate specificity and sensitivity
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)


# Calculate precision, recall and F1-score
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)

# calculate FPR and TPR
fpr, tpr, thresholds = roc_curve(true_labels, predicted_labels)

# calculate AUCROC
auc_roc = auc(fpr, tpr)



print("Specificity:", specificity)
print("Sensitivity:", sensitivity)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("F1-score:", f1)
print("AUCROC score: {:.3f}".format(auc_roc))

Specificity: 0.6428314337132573
Sensitivity: 0.45605208641609946
Accuracy: 0.5889533891070514
Precision: 0.34108012394864984
F1-score: 0.390274787894137
AUCROC score: 0.549


In [12]:
# Generate classification report
report = classification_report(true_labels, predicted_labels, labels=[0, 1], digits=4,zero_division="warn")

# Print report
print(report)



              precision    recall  f1-score   support

           0     0.8319    0.6296    0.7167      9508
           1     0.2205    0.4515    0.2963      2206

    accuracy                         0.5960     11714
   macro avg     0.5262    0.5405    0.5065     11714
weighted avg     0.7167    0.5960    0.6375     11714



In [18]:
# calculate FPR and TPR
fpr, tpr, thresholds = roc_curve(true_labels, predicted_labels)

# calculate AUCROC
auc_roc = auc(fpr, tpr)

# print the result
print("AUCROC score: {:.3f}".format(auc_roc))

AUCROC score: 0.541


In [16]:
accuracy = accuracy_score(true_labels, predicted_labels)
class_1_accuracy = accuracy_score([1 if label == 1 else 0 for label in true_labels], 
                                  [1 if label == 1 else 0 for label in predicted_labels])
class_0_accuracy = accuracy_score([1 if label == 0 else 0 for label in true_labels], 
                                  [1 if label == 0 else 0 for label in predicted_labels])
# class_minus_1_accuracy = accuracy_score([1 if label == -1 else 0 for label in true_labels], 
#                                          [1 if label == -1 else 0 for label in predicted_labels])

print("Overall accuracy:", accuracy)
print("Class 1 accuracy:", class_1_accuracy)
print("Class 0 accuracy:", class_0_accuracy)
# print("Class -1 accuracy:", class_minus_1_accuracy)

Overall accuracy: 0.5960389277787264
Class 1 accuracy: 0.5960389277787264
Class 0 accuracy: 0.5960389277787264
